<a href="https://colab.research.google.com/github/utkarshminhas/violence-detection-fyp/blob/main/notebooks/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set dataset paths

In [22]:
raw_rwf_dir = '/content/drive/Shareddrives/Final Year Project/Datasets/Mini-RWF-2000'
processed_rwf_dir = '/content/drive/Shareddrives/Final Year Project/Datasets/Processed'

# Export to frames

In [26]:
import os
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from tqdm import tqdm
import numpy as np

In [ ]:
def generate_base_folders():
    if not os.path.exists(processed_rwf_dir):
        os.makedirs(processed_rwf_dir)

    categories = os.listdir(raw_rwf_dir)
    
    for category in categories:
        cur_op_dir = os.path.join(processed_rwf_dir, category)
        print(category, cur_op_dir)
        if not os.path.exists(cur_op_dir):
            os.makedirs(cur_op_dir)

def convert_to_frames():
    categories = os.listdir(raw_rwf_dir)

    for category in categories:
        videos = os.listdir(os.path.join(raw_rwf_dir, category))

        for video in tqdm(videos):
            generate_frames(category, video)


fgbg = cv2.createBackgroundSubtractorMOG2(
        varThreshold=15,
    detectShadows=True
)


def get_mask(frame):
    mog_mask = fgbg.apply(frame)
    median_blur_mask = cv2.medianBlur(mog_mask, 5)
    bilateral_filter_mask = cv2.bilateralFilter(median_blur_mask, 9, 75, 75)
    gaussian_blur_mask = cv2.GaussianBlur(bilateral_filter_mask, (13, 13), 5)

    return gaussian_blur_mask


def generate_frames(category, video):
    base_video_name, _ = os.path.splitext(video)
    video_ip_path = os.path.join(raw_rwf_dir, category, video)
    video_op_dir_path = os.path.join(processed_rwf_dir, category, base_video_name)

    if not os.path.exists(video_op_dir_path):
        os.makedirs(video_op_dir_path)

    cap = cv2.VideoCapture(video_ip_path)
    framerate = int(cap.get(5))

    capture_frequency = 5

    while True:
        frame_number = int(cap.get(1))
        success, frame = cap.read()

        if not success or frame is None:
            break

        masked_frame = get_mask(frame)

        if frame_number % capture_frequency == 1:
            exporting_frame_number = int(frame_number / capture_frequency)
            frame_path = os.path.join(video_op_dir_path, "frame_{0:0=3d}.jpg".format(exporting_frame_number))
            cv2.imwrite(frame_path, masked_frame )

        k = cv2.waitKey(30)

        if k == 27 or k == ord('q'):
            break

        cap.release()


In [ ]:
with tf.device('/device:CPU:0'):
    convert_to_frames()

# Preprocessing

In [ ]:
# lalalal

# Model

## Training and validation generator

In [51]:
def preprocess(img):
    img = cv2.resize(img, (128, 128))
    img = img / 255.0
    return img

def get_data(batch_size):
    batch = [] # N elements

    label_to_int = {
        'Fight': 1,
        'NonFight': 0
    }

    categories = os.listdir(processed_rwf_dir)

    for category in categories:
        videos = os.listdir(os.path.join(processed_rwf_dir, category))

        for video in videos:
            frames = os.listdir(os.path.join(processed_rwf_dir, category, video))

            current_video = []

            for frame in frames:
                frame_path = os.path.join(processed_rwf_dir, category, video, frame)
                img = cv2.imread(frame_path)
                img = preprocess(img)
                current_video.append(img)

                # print(img.shape, len(current_video))

            current_video = np.array(current_video)
            batch.append(current_video)

            if len(batch) == batch_size:
                batch = np.array(batch)
                print(batch.shape)
                yield batch
                batch = []



def data_generator():
    pass

cur = next(get_data(2))
# print(cur.shape)

(2, 30, 128, 128, 3)


In [ ]:
image_size = (128, 128)

ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    processed_rwf_dir,
    labels="inferred",
    label_mode="binary",
    color_mode="grayscale",
    batch_size=seq_len,
    image_size=image_size,
    subset='training',
    validation_split=0.2,
    seed=1
)

ds_val = tf.keras.preprocessing.image_dataset_from_directory(
    processed_rwf_dir,
    labels="inferred",
    label_mode="binary",
    color_mode="grayscale",
    batch_size=seq_len,
    image_size=image_size,
    subset='validation',
    validation_split=0.2,
    seed=1
)

Found 360 files belonging to 2 classes.
Using 288 files for training.
Found 360 files belonging to 2 classes.
Using 72 files for validation.


In [ ]:
print(ds_train)

# for image_batch, labels_batch in ds_train:
#     print(image_batch.shape)
#     print(labels_batch.shape)
#     break

<BatchDataset shapes: ((None, 128, 128, 1), (None, 1)), types: (tf.float32, tf.float32)>


# Model

## Imports

In [ ]:
from tensorflow.keras import layers
import numpy as np
from keras.layers import TimeDistributed, Conv2D, Dense, MaxPooling2D, \
Flatten, LSTM, Dropout, BatchNormalization
from keras import models

import keras
from keras import optimizers
import tensorflow_addons as tfa

## Architecture

In [ ]:
input_shape = (seq_len, image_size[0], image_size[1], 1)

In [ ]:
model = models.Sequential(
    [
        TimeDistributed(
            Conv2D(64, (3, 3), activation=tf.nn.relu), 
            input_shape=input_shape
        ),
        TimeDistributed(MaxPooling2D((2, 2), strides=(1, 1))),
        TimeDistributed(Conv2D(128, (4, 4), activation=tf.nn.relu)),
        TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))),
        TimeDistributed(Conv2D(256, (4, 4), activation=tf.nn.relu)),
        TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))),
        TimeDistributed(Flatten()),
        Dropout(0.5),
        LSTM(256, return_sequences=False, dropout=0.5),
        Dense(2, activation=tf.nn.sigmoid)
    ]
)

model.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_175 (TimeDi (None, 30, 126, 126, 64)  640       
_________________________________________________________________
time_distributed_176 (TimeDi (None, 30, 125, 125, 64)  0         
_________________________________________________________________
time_distributed_177 (TimeDi (None, 30, 122, 122, 128) 131200    
_________________________________________________________________
time_distributed_178 (TimeDi (None, 30, 61, 61, 128)   0         
_________________________________________________________________
time_distributed_179 (TimeDi (None, 30, 58, 58, 256)   524544    
_________________________________________________________________
time_distributed_180 (TimeDi (None, 30, 29, 29, 256)   0         
_________________________________________________________________
time_distributed_181 (TimeDi (None, 30, 215296)      

In [ ]:
tqdm_callback = tfa.callbacks.TQDMProgressBar()
model_path = 'cnn_lstm_model.h5'

callbacks_list = [
    tqdm_callback,
    keras.callbacks.EarlyStopping(monitor=['acc'], patience=3),
    keras.callbacks.ModelCheckpoint(
        filepath=model_path,
        monitor='val_loss',
        save_best_only=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.1,
        patience=3
    )
]

optimizer=optimizers.RMSprop(lr=0.06)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])

In [ ]:
history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=1,
    # callbacks=callbacks_list,
)

ValueError: ignored